<a href="https://colab.research.google.com/github/Jahid1990/Product-Recommendation-Engine-Using-Deep-Learning/blob/master/pack_reco_engine_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load required packes and libraries

In [ ]:
#import cx_Oracle
import pandas as pd
import keras
import numpy as np
from keras import layers
from keras.layers import Input, LSTM, Dense,Conv1D,MaxPooling1D,Flatten,Embedding
from keras.layers import TimeDistributed
from keras.models import Model
from keras import metrics
from sklearn import preprocessing
import pickle

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Saving & loading data to/from PC

In [ ]:
# with open(r'training_data_pack_reco_new.pickle', 'wb') as f:
#      pickle.dump([x_train,y_train,x_test,y_test], f)

In [ ]:
with open(r'/content/drive/My Drive/training_data_pack_reco_new_with_rcg_info.pickle', 'rb') as f:
    [x_train,y_train,x_test,y_test]= pickle.load(f)

# Model Building

In [ ]:
from keras.layers import Concatenate

inputs = Input(shape=(x_train.shape[1],))
shared=Dense(1024,activation='relu')(inputs)
shared=Dense(1024,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)

pack=Dense(512,activation='relu')(shared)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='tanh')(pack)
pack=Dense(16,activation='softmax',name='pack')(pack)

day=Dense(512,activation='relu')(shared)
day=Dense(512,activation='relu')(day)
day=Dense(512,activation='relu')(day)
day=Dense(512,activation='relu')(day)
day=Dense(256,activation='relu')(day)
day=Dense(128,activation='tanh')(day)
day=Dense(32,activation='softmax',name='day')(day)


model=Model(inputs=inputs,outputs=[pack,day])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 126)          0                                            
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 1024)         130048      input_2[0][0]                    
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 1024)         1049600     dense_25[0][0]                   
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 512)          524800      dense_26[0][0]                   
____________________________________________________________________________________________

In [ ]:
from keras import optimizers
adm=optimizers.Adam( lr=0.001,beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
checkpoint = ModelCheckpoint('/content/drive/My Drive/pack_reco_engine_new_with_rcg_info.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

In [ ]:
#[x_test,[y_test[:,:16],y_test[:,16:]]]
model.compile(loss=['categorical_crossentropy','categorical_crossentropy'],optimizer='Adam', metrics=['accuracy'])
model.fit(x_train, [y_train[:,:16],y_train[:,16:]] ,epochs=100,validation_split=.2,verbose=1,batch_size=512,callbacks=[checkpoint])

Train on 1794568 samples, validate on 448642 samples
Epoch 1/100
1794568/1794568 [==============================] - 72s 40us/step - loss: 4.0543 - pack_loss: 1.6483 - day_loss: 2.4058 - pack_accuracy: 0.4848 - day_accuracy: 0.3792 - val_loss: 4.0046 - val_pack_loss: 1.6141 - val_day_loss: 2.3905 - val_pack_accuracy: 0.4930 - val_day_accuracy: 0.3839

Epoch 00001: val_loss improved from inf to 4.00463, saving model to /content/drive/My Drive/pack_reco_engine_new_with_rcg_info.h5
Epoch 2/100
1794568/1794568 [==============================] - 72s 40us/step - loss: 3.7799 - pack_loss: 1.4554 - day_loss: 2.3247 - pack_accuracy: 0.5401 - day_accuracy: 0.3945 - val_loss: 3.7346 - val_pack_loss: 1.4460 - val_day_loss: 2.2885 - val_pack_accuracy: 0.5466 - val_day_accuracy: 0.4021

Epoch 00002: val_loss improved from 4.00463 to 3.73463, saving model to /content/drive/My Drive/pack_reco_engine_new_with_rcg_info.h5
Epoch 3/100
1794568/1794568 [==============================] - 71s 40us/step - loss

In [ ]:
import numpy as np
import tensorflow as tf

np.set_printoptions(suppress=True)
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/pack_reco_engine_new_with_rcg_info.h5')

p=loaded_model.predict(x_test)

In [ ]:
pd.set_option('display.max_columns', 500)
pred=pd.DataFrame(p[0])
pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.347146,0.161935,0.276634,0.019264,0.079880,0.069738,0.011972,0.001433,0.010734,0.008876,0.004852,0.001433,0.003838,0.000318,0.000679,0.001267
1,0.031442,0.004140,0.000170,0.000277,0.027355,0.002337,0.008687,0.005085,0.002028,0.882822,0.000674,0.008742,0.001381,0.001892,0.004049,0.018919
2,0.016540,0.005350,0.002333,0.012780,0.002441,0.950620,0.000326,0.000068,0.004086,0.001015,0.001064,0.000111,0.003213,0.000005,0.000010,0.000038
3,0.401096,0.038112,0.002780,0.002983,0.404609,0.014903,0.029172,0.009623,0.006555,0.055325,0.001425,0.005729,0.004697,0.003404,0.004727,0.014860
4,0.491298,0.022056,0.003313,0.004215,0.379231,0.020764,0.023679,0.003710,0.009469,0.025639,0.001924,0.002487,0.005327,0.000919,0.001799,0.004170


# Select TOP 3 

In [ ]:
top3=np.argsort(-p[0])[:,:3]
#pd.DataFrame(top3)
ytest=np.argmax(y_test[:,:16],axis=1)

val=[]
for i in range(y_test.shape[0]):
  if ytest[i] in top3[i]:
    val.append(1)
  else:
    val.append(0)

In [ ]:
pd.Series(val).value_counts()/(221887+27359)

1    0.890233
0    0.109767
dtype: float64

In [ ]:
pd.crosstab(val,ytest)

In [ ]:
pred_class=np.argmax(p[0], axis=1)

In [ ]:
actual=np.argmax(y_test[:,:16],axis=1)

In [ ]:
pd.crosstab(pred_class,actual)

col_0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
row_0,,,,,,,,,,,,,,,,
0,58266,6307,788,1070,7354,4658,1614,347,1837,2955,577,425,2049,384,365,1205
1,1784,6885,227,121,293,502,41,1,124,23,28,5,35,0,0,2
2,168,139,846,34,33,89,6,1,16,1,2,0,4,2,1,0
3,987,386,64,3702,141,886,28,4,74,27,20,0,52,4,1,3
4,7721,1799,249,347,19828,1276,1752,130,332,1405,71,127,185,83,109,307
5,5643,2761,735,1991,761,30519,164,25,857,196,320,21,531,14,14,32
6,827,219,25,21,859,155,2701,6,57,242,17,94,41,2,3,16
7,87,11,1,1,39,4,7,246,0,69,0,0,2,7,11,18
8,1223,371,307,194,331,1086,72,14,7475,174,281,42,336,3,8,26


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(val,ytest))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.01      0.04      0.02     28292
           1       0.79      0.07      0.13    221708
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0

    accuracy                           0.07    250000
   macro avg       0.05   

In [ ]:
act=np.argmax(y_test[:,:16],axis=1)
pd.Series(act).value_counts()

0     84289
5     40686
4     30867
1     19502
12    15828
8     11855
9     10503
3      7758
6      6803
15     6774
10     4267
2      3240
14     2469
13     1962
11     1332
7      1111
dtype: int64

In [ ]:
pd.Series(pred_class).value_counts()

0     102662
5      35782
4      29786
12     15693
1      14401
8      12617
15      9488
9       8658
3       6649
6       5386
10      3250
2       1567
13      1526
11      1094
7        573
14       114
dtype: int64